In [1]:
################################################################################
################# Do not change the code in this cell ##########################
################################################################################

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import tqdm


class Net(nn.Module):
    def __init__(self, hidden_dim):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, hidden_dim, 3, 1)
        self.conv2 = nn.Conv2d(hidden_dim, 64, 3, 1)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

train_set = datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
test_set = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=True)

################################################################################
################# Do not change the code in this cell ##########################
################################################################################

In [ ]:
def train_for_five_epochs(model, optimizer, dataloader):
    model.train()
    for epoch in range(5):
        for imgs, labels in tqdm.tqdm(dataloader):
            imgs, labels = imgs.cuda(), labels.cuda()
            out = model(imgs)
            loss = F.nll_loss(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def test(model, dataloader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for imgs, labels in tqdm.tqdm(dataloader):
            imgs, labels = imgs.cuda(), labels.cuda()
            out = model(imgs)
            predictions = out.argmax(dim=1, keepdim=True)  
            correct += predictions.eq(labels.view_as(predictions)).sum().item()
    print(correct/len(dataloader.dataset))


hidden_dim    = 32
learning_rate = 0.01
momentum      = 0.9

network = Net(hidden_dim).cuda()
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)


train_for_five_epochs(network, optimizer, train_loader)
test(network, test_loader)